<a href="https://colab.research.google.com/github/forumb94/energypricecode/blob/main/maincode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import io

In [7]:
from google.colab import files
uploaded = files.upload()
df = pd.read_csv('final_merged_data.csv')

Saving final_merged_data.csv to final_merged_data.csv


In [8]:
# Read the dataset
df = pd.read_csv(io.BytesIO(uploaded['final_merged_data.csv']))

In [9]:
# Convert 'Time' column to datetime format
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M', errors='coerce')

# Fill missing values for numeric columns using forward fill, then backward fill as backup
numeric_cols = df.select_dtypes(include=["number"]).columns
df[numeric_cols] = df[numeric_cols].ffill().bfill()

# Drop unnecessary columns
df.drop(columns=['Local Date', 'Hour Number'], inplace=True, errors='ignore')

# Display dataset info
print(df.info())

# Display first few rows
print(df.head())

# Generate summary statistics
summary_stats = df.describe().T
missing_values = df.isnull().sum()
unique_counts = df.nunique()

# Display key statistics
print("\nSummary Statistics:")
print(summary_stats)

print("\nMissing Values Per Column:")
print(missing_values[missing_values > 0])

print("\nUnique Value Counts for Categorical Columns:")
print(unique_counts)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35081 entries, 0 to 35080
Data columns (total 44 columns):
 #   Column                                                Non-Null Count  Dtype         
---  ------                                                --------------  -----         
 0   Time                                                  0 non-null      datetime64[ns]
 1   Total Generation (MW)                                 35081 non-null  float64       
 2   Dual Fuel Generation (MW)                             35081 non-null  float64       
 3   Hydro Generation (MW)                                 35081 non-null  float64       
 4   Natural Gas Generation (MW)                           35081 non-null  float64       
 5   Nuclear Generation (MW)                               35081 non-null  float64       
 6   Other Fossil Fuels Generation (MW)                    35081 non-null  float64       
 7   Other Renewables Generation (MW)                      35081 non-null  float6